# Guía 3 — Introducción a LangChain con OpenAI API
Curso: **IA en el Aula — Nivel Avanzado**  
Profesor: **Luis Daniel Benavides Navarro**  
Estudiante: **Juan Esteban Medina Rivas**  
Fecha: **Octubre 2025**

En esta guía aprenderás los conceptos básicos de **LangChain**, una librería diseñada para construir aplicaciones impulsadas por modelos de lenguaje, integrando la API de OpenAI con flujos más complejos. Exploraremos cómo crear un primer **prompt chain**, administrar memoria y usar herramientas para componer respuestas inteligentes.

## 1️⃣ Instalación y configuración inicial
LangChain se instala como cualquier otra librería de Python. También usaremos `python-dotenv` para gestionar la clave de OpenAI. Ejecute la siguiente celda:

In [7]:
%pip install openai langchain python-dotenv langchain-openai

Note: you may need to restart the kernel to use updated packages.


## 2️⃣ Cargar variables de entorno y cliente OpenAI
Crea un archivo `.env` con tu clave de API y cárgala para poder usarla dentro de LangChain. Esto evita exponer tu clave en el código fuente.

In [8]:

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True)  # Cargar archivo .env
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("No se encontró la clave OPENAI_API_KEY en el archivo .env")

# Crear cliente LangChain con OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
print("Cliente LangChain con OpenAI inicializado correctamente.")


Cliente LangChain con OpenAI inicializado correctamente.


## 3️⃣ Primer ejemplo: Prompt simple con LangChain
LangChain utiliza **chains** (cadenas de pasos) para procesar información. Comencemos con una cadena simple que envía un mensaje al modelo y obtiene la respuesta.

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the LLM (uses your OpenAI API key from environment)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Create a simple prompt
prompt = ChatPromptTemplate.from_template(
    "Explica en dos frases el concepto de {tema}."
)

# Combine the components using LCEL (LangChain Expression Language)
chain = prompt | llm | StrOutputParser()

# Run it
result = chain.invoke({"tema": "aprendizaje automático"})
print(result)


El aprendizaje automático es una rama de la inteligencia artificial que permite a las computadoras aprender de datos y mejorar su rendimiento en tareas específicas sin ser programadas explícitamente para ello. Utiliza algoritmos para identificar patrones y hacer predicciones basadas en la información recibida.


### Explicación
- **ChatPromptTemplate:** define la estructura del mensaje que se envía al modelo.
- **ChatOpenAI:** la conexión real al modelo.
- **chain** crea la cadena de componentes usando LCEL (LangChain Expression Language).
- **StrOutputParser** convierte la salida estructurada del modelo en texto plano.
- **chain.invoke:** ejecuta la cadena pasando los valores del prompt.

Este patrón permite reutilizar prompts para distintos temas o contextos.

## 4️⃣ Ejemplo 2: Encadenar múltiples pasos
LangChain permite combinar varios pasos en una misma ejecución. En este ejemplo, crearemos dos prompts: uno para definir un tema y otro para generar una aplicación educativa basada en el resultado.

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o")

# Primer prompt
primer_prompt = PromptTemplate(
    input_variables=["tema"],
    template="Explica brevemente el concepto de {tema}."
)

# Segundo prompt
segundo_prompt = PromptTemplate(
    input_variables=["concepto"],
    template="Propón una aplicación educativa basada en: {concepto}."
)

parser = StrOutputParser()

# Cadena moderna usando LCEL
cadena = primer_prompt | llm | segundo_prompt | llm | parser

# Ejecutar
resultado = cadena.invoke({"tema": "realidad aumentada"})
print(resultado)

Una aplicación educativa innovadora utilizando realidad aumentada (AR) podría ser "ARquitectura del Saber", diseñada para estudiantes de arquitectura, historia o arte. Esta aplicación permitiría a los usuarios explorar y aprender sobre estructuras históricas y arquitectónicas famosas en todo el mundo, directamente desde su dispositivo móvil.

### Características principales:

1. **Exploración Virtual:**
   - Los estudiantes pueden colocar modelos en 3D de edificios históricos en su entorno real. Estos modelos pueden escalarse para observar detalles arquitectónicos de cerca.

2. **Información Interactiva:**
   - Al acercarse a diferentes partes de la estructura virtual, se desplegaría información detallada sobre su historia, estilo arquitectónico y técnicas de construcción, con audios y textos.

3. **Reconocimiento de Ambientes Reales:**
   - Al apuntar con el dispositivo hacia ciertos edificios o monumentos reales, la aplicación proporcionaría información contextual y comparativa, enri

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# 1) LLM (usa tu OPENAI_API_KEY en el entorno)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
to_str = StrOutputParser()

# 2) Paso 1: explicar brevemente el concepto de {tema}
primer_prompt = ChatPromptTemplate.from_template(
    "Explica brevemente el concepto de {tema}."
)
primer_paso = primer_prompt | llm | to_str
# `primer_paso` produce un string, por ejemplo: "La realidad aumentada es ..."

# 3) Paso 2: proponer una aplicación educativa usando la salida del paso 1 como {concepto}
segundo_prompt = ChatPromptTemplate.from_template(
    "Propón una aplicación educativa del siguiente concepto: {concepto}."
)
segundo_paso = segundo_prompt | llm | to_str

# 4) Encadenar: mapear la entrada {tema} al primer paso, y su salida a {concepto} del segundo
cadena_secuencial = {"concepto": primer_paso} | segundo_paso

# 5) Ejecutar la cadena completa
resultado = cadena_secuencial.invoke({"tema": "realidad aumentada"})
print(resultado)

### Aplicación Educativa: "Aventuras Históricas en Realidad Aumentada"

**Descripción de la Aplicación:**
"Aventuras Históricas" es una aplicación educativa que utiliza la realidad aumentada para llevar a los estudiantes a un viaje interactivo a través de diferentes períodos históricos y eventos significativos. La aplicación permite a los usuarios explorar lugares históricos, interactuar con figuras históricas y participar en actividades educativas que enriquecen su comprensión del pasado.

**Características Clave:**

1. **Exploración de Sitios Históricos:**
   - Los estudiantes pueden apuntar sus dispositivos hacia un lugar específico (por ejemplo, un monumento, un edificio o un parque) y ver representaciones en 3D de cómo era ese lugar en diferentes épocas. Por ejemplo, al apuntar hacia una antigua fortaleza, pueden ver cómo se veía en su apogeo.

2. **Interacción con Figuras Históricas:**
   - La aplicación presenta avatares de figuras históricas que pueden "cobrar vida" en el entor

### Explicación
- **cadena_secuencial:** permite conectar varias cadenas; la salida de una se convierte en la entrada de la siguiente.
- En este caso, el modelo primero explica el tema y luego sugiere una aplicación educativa.

Este tipo de flujo es ideal para generar contenido didáctico o ideas para proyectos en clase.

## 5️⃣ Ejemplo 3: Añadir memoria a la conversación
LangChain incluye módulos de **memoria** para mantener contexto entre múltiples interacciones. Esto permite simular conversaciones educativas más naturales, donde el modelo recuerda temas previos.

In [12]:

# Instalar si hace falta:
# %pip install -U langchain langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

# LLM (requiere OPENAI_API_KEY en el entorno)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
to_str = StrOutputParser()

# Prompt con hueco para el historial
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente educativo claro y conciso."),
    MessagesPlaceholder("chat_history"),      # ← aquí va la memoria
    ("human", "{input}")
])

# Cadena base
chain = prompt | llm | to_str

# Memoria simple como lista de mensajes
history: list = []

def chat(user_text: str) -> str:
    """
    Envía un turno del usuario, usa el historial y actualiza la memoria
    con el par (usuario, asistente).
    """
    global history
    # Ejecutar la cadena inyectando el historial actual
    answer = chain.invoke({"input": user_text, "chat_history": history})
    # Actualizar memoria (guardar los dos mensajes)
    history += [HumanMessage(content=user_text), AIMessage(content=answer)]
    return answer

# --- Ejemplo de uso (tres turnos) ---
print(chat("Hola, soy un profesor de informática."))
print(chat("¿Puedes explicarme cómo introducir IA a mis estudiantes?"))
print(chat("¿Qué ejemplos prácticos puedo usar en la clase?"))



¡Hola! ¿En qué puedo ayudarte hoy con respecto a la informática?
Claro, aquí tienes algunas ideas sobre cómo introducir la inteligencia artificial (IA) a tus estudiantes:

1. **Conceptos Básicos**:
   - Comienza explicando qué es la IA y sus diferentes ramas, como el aprendizaje automático, el procesamiento del lenguaje natural y la visión por computadora.
   - Usa ejemplos cotidianos, como asistentes virtuales (Siri, Alexa) y recomendaciones de productos.

2. **Historia y Evolución**:
   - Presenta una breve historia de la IA, desde sus inicios hasta los avances recientes.
   - Discute hitos importantes, como el juego de ajedrez de Deep Blue y el desarrollo de redes neuronales.

3. **Aplicaciones Prácticas**:
   - Muestra aplicaciones de IA en diversas industrias: salud, finanzas, transporte, entretenimiento, etc.
   - Invita a los estudiantes a investigar y presentar sobre aplicaciones específicas que les interesen.

4. **Proyectos Prácticos**:
   - Propón proyectos simples, como cre

### Explicación
- **ConversationBufferMemory:** almacena los mensajes anteriores en la conversación.
- **ConversationChain:** combina el modelo con la memoria.
- Esta funcionalidad es útil para tutores inteligentes o chatbots educativos que requieren continuidad en el diálogo.

## 6️⃣ Buenas prácticas con LangChain + OpenAI
- Usa prompts **claros y estructurados**: el modelo responde mejor cuando la tarea está bien definida.
- Controla `temperature` según la tarea: bajo (0.1–0.3) para precisión, alto (0.7–0.9) para creatividad.
- Guarda logs o historiales si tu aplicación incluye interacción prolongada.
- Limita la longitud de las respuestas con `max_tokens` para evitar costos o respuestas excesivas.
- Documenta tus cadenas (`LLMChain`) para reusarlas en distintos contextos educativos.

## ✅ Conclusión
Has aprendido los fundamentos de LangChain: prompts, chains, memoria y flujo secuencial. Estos conceptos son la base para desarrollar asistentes educativos, tutores personalizados o sistemas de generación de contenido en el aula. En la próxima guía implementaremos un **asistente educativo con RAG (Retrieval-Augmented Generation)** usando tus propios materiales docentes.